In [47]:
import pandas as pd
import numpy as np
import sympy as sp
from scipy.optimize import differential_evolution,NonlinearConstraint, Bounds, LinearConstraint

In [52]:
df=pd.read_csv('amzn_price.csv')
df.drop('Unnamed: 0',inplace=True,axis=1)

In [53]:
df

,ma30_50_signal,ma30_200_signal,ma50_200_signal,Obv_signal,sma100_signal,macd_signal,adx_signal,stoch_signal,roc_signal,rsi_signal,cci_signal,atr_signal,wpr_signal,bb_signal,Close
0,1,1,1,1,1,1,1,0,1,0,0,-1,0,0,19.898500
1,0,0,0,-1,1,0,0,0,0,0,0,0,0,0,19.822001
2,0,0,0,0,1,0,0,1,-1,0,0,0,0,0,19.681499
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,19.901501
4,0,0,0,1,1,0,0,0,0,0,0,0,-1,0,20.096001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,0,0,0,0,1,0,0,1,0,0,0,0,0,0,58.417999
1003,0,0,0,0,1,0,0,0,0,0,0,0,0,0,58.838001
1004,0,0,0,0,1,0,0,0,0,0,0,0,0,0,59.112999
1005,0,0,0,0,1,0,0,0,0,0,0,0,0,0,59.305000


In [54]:
df_amzn_train=df

In [55]:
sl=[]
for i in range (1,1008):
    s_i=df_amzn_train.iloc[i-1]
    sl.append(s_i)
sl

[ma30_50_signal      1.0000
 ma30_200_signal     1.0000
 ma50_200_signal     1.0000
 Obv_signal          1.0000
 sma100_signal       1.0000
 macd_signal         1.0000
 adx_signal          1.0000
 stoch_signal        0.0000
 roc_signal          1.0000
 rsi_signal          0.0000
 cci_signal          0.0000
 atr_signal         -1.0000
 wpr_signal          0.0000
 bb_signal           0.0000
 Close              19.8985
 Name: 0, dtype: float64,
 ma30_50_signal      0.000000
 ma30_200_signal     0.000000
 ma50_200_signal     0.000000
 Obv_signal         -1.000000
 sma100_signal       1.000000
 macd_signal         0.000000
 adx_signal          0.000000
 stoch_signal        0.000000
 roc_signal          0.000000
 rsi_signal          0.000000
 cci_signal          0.000000
 atr_signal          0.000000
 wpr_signal          0.000000
 bb_signal           0.000000
 Close              19.822001
 Name: 1, dtype: float64,
 ma30_50_signal      0.000000
 ma30_200_signal     0.000000
 ma50_200_signal  

In [56]:
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')

In [57]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10 = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10

In [58]:
nlc = NonlinearConstraint(const, 1.0 ,1.0)

In [69]:
bounds = [[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0],[0.0, 1.0], [0.0, 1.0]]

In [70]:
def func(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10 = p
    mon=100000
    td=0.3
    amount=0
    for i in range(0,len(df_amzn_train)):
        r = ((w1*sl[i][0]+w2*sl[i][1]+w3*sl[i][2]+w4*sl[i][3]+w5*sl[i][4]+w6*sl[i][5]+w7*sl[i][6]+w8*sl[i][7]+w9*sl[i][8]+w10*sl[i][9])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))
        if r>td:
            sumall=mon//df_amzn_train['Close'][i]
            amount+=sumall
            mon-=sumall*df_amzn_train['Close'][i]
        elif r<-td:
            mon+=amount*df_amzn_train['Close'][i]
            amount=0
        else:
            pass
    ret=100000/mon
    return ret

In [71]:
df_amzn_train['Close'][0]

19.898500442504883

In [72]:
def fobj():
    result = differential_evolution(func, bounds,constraints=(nlc))
    print(result) 
    result = differential_evolution(func, bounds,constraints=(nlc), disp=True)

In [ ]:
fobj()

C:\Users\ASUS\anaconda3\lib\site-packages\scipy\optimize\_differentialevolution.py:898: UserWarning: differential evolution didn't find a solution satisfying the constraints, attempting to polish from the least infeasible solution
  warnings.warn("differential evolution didn't find a"
C:\Users\ASUS\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


           constr: [array([0.])]
 constr_violation: 0.0
              fun: 12572.892913095413
              jac: [array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]), array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])]
            maxcv: 0.0
          message: 'Maximum number of iterations has been exceeded.'
             nfev: 66
              nit: 1000
          success: False
                x: array([0.11579913, 0.11579211, 0.03264646, 0.0331518 , 0.03463367,
       0.03430722, 0.20774563, 0.06661848, 0.16458681, 0.1947187 ])
differential_evolution ste